# Import Required Libraries
Import the necessary libraries, including NumPy, Pandas, and Bokeh.

In [7]:
# Import Required Libraries
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, ColorBar
from bokeh.transform import linear_cmap
from bokeh.layouts import column
from bokeh.io import push_notebook
from bokeh.palettes import Viridis256
from datetime import datetime, timedelta

# Enable Bokeh to display plots in the notebook
output_notebook()

Loading BokehJS ...

# Define Constants and Locations
Define the constants and locations used in the calculations, such as Earth's orbital eccentricity and the coordinates of specific locations.

In [8]:
# Define Constants and Locations

# Constants
e = 0.0167  # Earth's orbital eccentricity
lambda_p = 102.9372  # Longitude of perihelion in degrees
y = 0.043  # Related to Earth's axial tilt and ellipticity
days_in_year = 365  # Number of days in a year

# Locations
locations = {
    "Tenerife": {"latitude": 28.1800, "longitude": -16.3035},  # Source: IAC webpage
    "MtKent": {"latitude": -28.0000, "longitude": 152.0000}  # Source: UNISQ webpage
}

# Equation of Time Calculations
Define functions to calculate the equation of time using both precise and simplified methods.

In [9]:
# Equation of Time Calculations

# Define functions to calculate the equation of time using both precise and simplified methods
def eot_precise(day_of_year):
    M = 2 * np.pi * (day_of_year - 3) / 365.24  # Mean anomaly of the Earth's orbit
    return -7.659 * np.sin(M) + 9.863 * np.sin(2 * M + np.radians(3.5932))  # Result in minutes

def eot_simplified(day_of_year):
    B = 2 * np.pi * (day_of_year - 81) / 365  # Mean anomaly of the Earth's orbit
    g = 2 * np.pi * (day_of_year - 81) / 365.24  # Mean anomaly of the Earth's orbit
    return 7.5 * np.sin(2 * B) - 9.87 * np.sin(4 * B) - 1.914 * np.sin(g)  # Result in minutes

# Generate data for plotting
data = []
for day in range(1, days_in_year + 1):
    precise = eot_precise(day)
    simplified = eot_simplified(day)
    data.append({"Day": day, "Precise": precise, "Simplified": simplified})

# Convert to DataFrame
df = pd.DataFrame(data)
df["Difference"] = df["Precise"] - df["Simplified"]

# Create ColumnDataSource for Bokeh
source = ColumnDataSource(df)

# Create Bokeh plot
p = figure(title="Comparison of Simplified and Precise Equation of Time", x_axis_label="Day of Year", y_axis_label="EoT (minutes)", width=800, height=400)
p.line(x="Day", y="Precise", source=source, legend_label="Precise EoT", line_width=2, color="blue")
p.line(x="Day", y="Simplified", source=source, legend_label="Simplified EoT", line_width=2, color="green", line_dash="dashed")
p.line(x="Day", y="Difference", source=source, legend_label="Difference (Precise - Simplified)", line_width=2, color="red", line_dash="dotdash")

# Add hover tool
hover = HoverTool()
hover.tooltips = [("Day", "@Day"), ("Precise EoT", "@Precise"), ("Simplified EoT", "@Simplified"), ("Difference", "@Difference")]
p.add_tools(hover)

# Show plot
show(p, notebook_handle=True)

# Declination and Daylight Calculations
Define functions to calculate the solar declination angle and daylight duration based on the day of the year and latitude.

In [10]:
# Declination and Daylight Calculations

# Define functions to calculate the solar declination angle and daylight duration based on the day of the year and latitude
def calculate_declination(day_of_year):
    return np.degrees(np.arcsin(np.sin(np.radians(-23.44)) * np.cos(2 * np.pi * (day_of_year + 10) / 365.24 + 2 * 0.0167 * np.sin(2 * np.pi * (day_of_year - 2) / 365.24))))

def calculate_daylight(lat, day_of_year):
    decl = np.radians(calculate_declination(day_of_year))
    lat_rad = np.radians(lat)
    hour_angle = np.arccos(-np.tan(lat_rad) * np.tan(decl))
    daylight_hours = 2 * np.degrees(hour_angle) / 15
    return daylight_hours

# Generate data for plotting
data = []
for day in range(1, days_in_year + 1):
    declination = calculate_declination(day)
    daylight_tenerife = calculate_daylight(locations["Tenerife"]["latitude"], day)
    daylight_mtkent = calculate_daylight(locations["MtKent"]["latitude"], day)
    data.append({"Day": day, "Declination": declination, "Daylight_Tenerife": daylight_tenerife, "Daylight_MtKent": daylight_mtkent})

# Convert to DataFrame
df_declination_daylight = pd.DataFrame(data)

# Create ColumnDataSource for Bokeh
source_declination_daylight = ColumnDataSource(df_declination_daylight)

# Create Bokeh plot for Declination
p_declination = figure(title="Solar Declination Angle", x_axis_label="Day of Year", y_axis_label="Declination (degrees)", width=800, height=400)
p_declination.line(x="Day", y="Declination", source=source_declination_daylight, legend_label="Declination", line_width=2, color="purple")

# Add hover tool for Declination plot
hover_declination = HoverTool()
hover_declination.tooltips = [("Day", "@Day"), ("Declination", "@Declination")]
p_declination.add_tools(hover_declination)

# Create Bokeh plot for Daylight Duration
p_daylight = figure(title="Daylight Duration", x_axis_label="Day of Year", y_axis_label="Daylight Hours", width=800, height=400)
p_daylight.line(x="Day", y="Daylight_Tenerife", source=source_declination_daylight, legend_label="Tenerife", line_width=2, color="blue")
p_daylight.line(x="Day", y="Daylight_MtKent", source=source_declination_daylight, legend_label="MtKent", line_width=2, color="green")

# Add hover tool for Daylight plot
hover_daylight = HoverTool()
hover_daylight.tooltips = [("Day", "@Day"), ("Daylight Tenerife", "@Daylight_Tenerife"), ("Daylight MtKent", "@Daylight_MtKent")]
p_daylight.add_tools(hover_daylight)

# Show plots
show(column(p_declination, p_daylight), notebook_handle=True)

# Generate Analemma Data
Generate data for the analemma, including the equation of time, declination, and daylight duration for each day of the year.

In [11]:

# Declination and Daylight Calculations
def calculate_declination(day_of_year):
    return np.degrees(np.arcsin(np.sin(np.radians(-23.44)) * np.cos(2 * np.pi * (day_of_year + 10) / 365.24 + 2 * 0.0167 * np.sin(2 * np.pi * (day_of_year - 2) / 365.24))))

def calculate_daylight(lat, day_of_year):
    decl = np.radians(calculate_declination(day_of_year))
    lat_rad = np.radians(lat)
    hour_angle = np.arccos(-np.tan(lat_rad) * np.tan(decl))
    daylight_hours = 2 * np.degrees(hour_angle) / 15
    return daylight_hours

# Generate Analemma Data
data = []
for day in range(1, days_in_year + 1):
    precise = eot_precise(day)
    declination = calculate_declination(day)
    daylight_tenerife = calculate_daylight(locations["Tenerife"]["latitude"], day)
    daylight_mtkent = calculate_daylight(locations["MtKent"]["latitude"], day)
    date = datetime(2023, 1, 1) + timedelta(days=day - 1)
    data.append({
        "Day": day,
        "Date": date.strftime("%B %d"),
        "Precise": precise,
        "Declination": declination,
        "Daylight_Tenerife": daylight_tenerife,
        "Daylight_MtKent": daylight_mtkent
    })

# Convert to DataFrame
df_analemma = pd.DataFrame(data)

# Create ColumnDataSource for Bokeh
source_analemma = ColumnDataSource(df_analemma)

# Create Color Mapper
color_mapper = linear_cmap(field_name='Day', palette=Viridis256, low=min(df_analemma['Day']), high=max(df_analemma['Day']))

# Create Bokeh plot for Analemma
p_analemma = figure(title="Analemma", x_axis_label="Equation of Time (minutes)", y_axis_label="Solar Declination (degrees)", width=800, height=400)
p_analemma.circle(x="Precise", y="Declination", source=source_analemma, size=8, color=color_mapper, alpha=0.6)

# Add hover tool for Analemma plot
hover_analemma = HoverTool()
hover_analemma.tooltips = [
    ("Day", "@Day"),
    ("Date", "@Date"),
    ("EoT", "@Precise"),
    ("Declination", "@Declination"),
    ("Daylight Tenerife", "@Daylight_Tenerife"),
    ("Daylight MtKent", "@Daylight_MtKent")
]
p_analemma.add_tools(hover_analemma)

# Add Color Bar
color_bar = ColorBar(color_mapper=color_mapper['transform'], width=8, location=(0,0))
p_analemma.add_layout(color_bar, 'right')

# Show plot
show(p_analemma, notebook_handle=True)